In [111]:
import sys
!{sys.executable} -m pip install essentia
!{sys.executable} -m pip install pymongo

     |████████████████████████████████| 462 kB 3.0 MB/s eta 0:00:01


In [112]:
import os
import essentia
import essentia.standard as es
import pymongo

In [55]:
#filename = "./../../music/16 Bit Lolitas/Beat Organ EP/01 Beat Organ (original mix).mp3"
filename = './../../music/Ne‐Yo/Year of the Gentleman/01 Closer.mp3'

features, features_frames = es.MusicExtractor(profile="profile.yml")(filename)

In [64]:
# scan all directories for audio files
directories = [x[0] for x in os.walk('./../../music/')]
for directory in directories:
    for file in os.listdir(directory):
        if file.endswith('.mp3') or file.endswith('.m4a') or file.endswith('.ogg') or file.endswith('.m4p') or file.endswith('.flac'):
            print(file)
            filename = directory+'/'+file
            features, features_frames = es.MusicExtractor(profile="profile.yml")(filename)
            print(features_frames)
            # TODO store in database
            raise

01 Doncamatic.mp3


RuntimeError: No active exception to reraise

In [54]:
#print(sorted(features.descriptorNames()))

In [109]:
ignorePrefix = [
    'lowlevel.barkbands',
    'lowlevel.dissonance',
    'lowlevel.erbbands',
    'lowlevel.melbands',
    'lowlevel.mfcc',
    'lowlevel.hfc',
    'lowlevel.loudness_ebu128',
    'lowlevel.pitch_salience',
    'lowlevel.silence_rate',
    'lowlevel.spectral_',
    'lowlevel.zerocrossingrate',
    'tonal.hpcp',
    'rhythm.beats_loudness',
    'rhythm.beats_position',
    'rhythm.bpm_histogram',
    'tonal.chords_histogram',
    'tonal.thpcp',
    'tonal.chords_strength',
    'tonal.tuning_diatonic_strength',
    'tonal.tuning_equal_tempered_deviation',
    'tonal.tuning_nontempered_energy_ratio',
    'tonal.key',
    'lowlevel.gfcc',
    'lowlevel.mfcc',
    'metadata'
]

ignoreSuffix = [
    '.probability',
    '.strength'
]

ignoreWildcard = [
    'not_'
]

whitelist = [
    'lowlevel.dissonance.median',
    'rhythm.beats_loudness.median',
    'tonal.chords_strength.median',
    'tonal.key_krumhansl.key',
    'tonal.key_krumhansl.scale'
]

for stat in features.descriptorNames():
    if stat in whitelist or (not 'not_' in stat and not stat.startswith(tuple(ignorePrefix)) and not stat.endswith(tuple(ignoreSuffix))):
    #if (not stat.startswith(tuple(ignorePrefix))):
        print(stat+':  '+str(features[stat]))

highlevel.danceability.all.danceable:  0.9999253153800964
highlevel.genre_electronic.all.ambient:  0.2901545763015747
highlevel.genre_electronic.all.dnb:  0.04171855375170708
highlevel.genre_electronic.all.house:  0.31343817710876465
highlevel.genre_electronic.all.techno:  0.017702948302030563
highlevel.genre_electronic.all.trance:  0.3369857370853424
highlevel.genre_rosamerica.all.cla:  0.001343433978036046
highlevel.genre_rosamerica.all.dan:  0.1654413789510727
highlevel.genre_rosamerica.all.hip:  0.013723010197281837
highlevel.genre_rosamerica.all.jaz:  0.0022148415446281433
highlevel.genre_rosamerica.all.pop:  0.6972100734710693
highlevel.genre_rosamerica.all.rhy:  0.08043279498815536
highlevel.genre_rosamerica.all.roc:  0.03649448603391647
highlevel.genre_rosamerica.all.spe:  0.003139983396977186
highlevel.mood_acoustic.all.acoustic:  0.020447153598070145
highlevel.mood_aggressive.all.aggressive:  0.30611512064933777
highlevel.mood_electronic.all.electronic:  0.3876411020755768
hi